In [1]:
import requests
from dataset import get_recipe
from prompt import example_prompt
import pandas as pd

In [24]:
url = 'http://127.0.0.1:8000/gemini/questions/en/'
url_desc = 'http://127.0.0.1:8000/gemini/description/en/'

In [15]:
recipe = get_recipe(1)

In [16]:
recipe_name = recipe['name']
ingredients = recipe['ingredients']
steps = recipe['steps']
time = recipe['minutes']

In [35]:
steps

"['preheat oven to 425 degrees f', 'press dough into the bottom and sides of a 12 inch pizza pan', 'bake for 5 minutes until set but not browned', 'cut sausage into small pieces', 'whisk eggs and milk in a bowl until frothy', 'spoon sausage over baked crust and sprinkle with cheese', 'pour egg mixture slowly over sausage and cheese', 's& p to taste', 'bake 15-20 minutes or until eggs are set and crust is brown']"

In [17]:
data = {
    'recipe_name' : recipe_name,
    'ingredients' : ingredients,
    'steps' : steps,
    'time' : time
}

In [21]:
response = requests.post(url=url, data=data)

In [22]:
response.status_code

200

In [23]:
response.json()

{'response': '<step 1>\n1. What type of dough is used as the base for this pizza?\n2. How thick should the dough be rolled out?\n3. What is the purpose of preheating the oven to 425 degrees Fahrenheit?\n</step 1>\n<step 2>\n1. What type of pizza sauce is used in this recipe?\n2. How much pizza sauce is used per pizza?\n3. What is the purpose of spreading the pizza sauce evenly over the crust?\n</step 2>\n<step 3>\n1. What type of cheese is used as the main topping for this pizza?\n2. How much cheese is used per pizza?\n3. What is the purpose of sprinkling the cheese evenly over the pizza sauce?\n</step 3>\n<step 4>\n1. What other toppings can be added to this pizza?\n2. How much of each topping should be added?\n3. What is the purpose of adding additional toppings?\n</step 4>\n<step 5>\n1. What is the purpose of baking the pizza in the preheated oven?\n2. How long should the pizza be baked at 425 degrees Fahrenheit?\n3. What is the purpose of rotating the pizza halfway through the baki

In [36]:
desc = requests.post(url=url_desc, data=data)

In [28]:
desc.status_code

200

In [37]:
desc.json()

{'description': 'Indulge in our one-of-a-kind Breakfast Pizza, a delightful fusion of flavors and textures. This unique morning treat features a crispy prepared pizza crust, generously topped with a savory combination of eggs, sausage patties, and mushrooms. A blanket of melted cheese provides a luscious, stretchy layer, while a sprinkling of fragrant basil adds a touch of freshness. Each slice offers a symphony of hearty flavors and textures, sure to awaken your taste buds and kick-start your day with a burst of energy.'}